In [1]:
import time
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

np.random.seed(42)
torch.manual_seed(42)

Definition of the architecture as seen in : https://arxiv.org/pdf/1811.08782.pdf

In [2]:
class DGM_layer(nn.Module):
    
    def __init__(self, in_features, out_feature, residual = False):
        super(DGM_layer, self).__init__()
        self.residual = residual
        
        self.Z = nn.Linear(out_feature,out_feature) ; self.UZ = nn.Linear(in_features,out_feature, bias=False)
        self.G = nn.Linear(out_feature,out_feature) ; self.UG = nn.Linear(in_features,out_feature, bias=False)
        self.R = nn.Linear(out_feature,out_feature) ; self.UR = nn.Linear(in_features,out_feature, bias=False)
        self.H = nn.Linear(out_feature,out_feature) ; self.UH = nn.Linear(in_features,out_feature, bias=False)
    

    def forward(self, x, s):
        z = torch.tanh(self.UZ(x)+self.Z(s))
        g = torch.tanh(self.UG(x)+self.G(s))
        r = torch.tanh(self.UR(x)+self.R(s))
        h = torch.tanh(self.UH(x)+self.H(s*r))
        return (1 - g) * h + z*s
        
    


In [3]:
class DGM_net(nn.Module):
    def __init__(self, in_dim,out_dim, n_layers, n_neurons, residual = False):
        """ in_dim is number of cordinates + 1 
            out_dim is the number of output
            n_layers and n_neurons are pretty self explanatory
            make residual = true for identity between each DGM layers
        """
        super(DGM_net, self).__init__()
        self.in_dim = in_dim ; self.out_dim = out_dim
        self.n_layers = n_layers
        self.n_neurons = n_neurons
        self.residual = residual

        self.first_layer = nn.Linear(in_dim, n_neurons)
        
        self.dgm_layers = nn.ModuleList([DGM_layer(self.in_dim, self.n_neurons,
                                                       self.residual) for i in range(self.n_layers)])
        self.final_layer = nn.Linear(n_neurons,out_dim)
    
    def forward(self,X,t):
        x = torch.cat((X,t),1)
        s = torch.tanh(self.first_layer(x))
        for i,dgm_layer in enumerate(self.dgm_layers):
            s = dgm_layer(x, s)
        
        return  self.final_layer(s)

In [4]:
k = 400 #thermal conductivity of copper
stefanBoltz = 5.670373e-8 # stefan-boltzman cst
h = 1 #plate size
Tb = 1000 #temperature of the bottom
epsilon=0.5 # emissivity
rho = 8960 # Copper density
Cp=386 #specific heat
Ta = 300 #ambient temperature
tz = 0.01 # plate thickness

#for ease of use
a = rho*Cp*tz
b = -k*tz
c = 2*h
d = 2*stefanBoltz*epsilon


# Time limits
T0 = 0.0 + 1e-10    # Initial time
T  = 5000.0            # Terminal time

# Space limits
S1 = 0.0 + 1e-10    # Low boundary
S2 = 1              # High boundary


# Training parameters
steps_per_sample = 10
sampling_stages = 800

# Number of samples
NS_1 = 500  #domain all t
NS_2 = 100   #dirichlet
NS_3 = 300   #neumann
NS_4 = 100   # initial


In [5]:
def sampler(N1, N2, N3, N4):
    # Sampler #1: PDE domain
    t1 = np.random.uniform(low=T0 - 0.5*(T - T0)  ,
                           high=T,
                           size=[N1,1])
    s1 = np.random.uniform(low=S1 - (S2 - S1)*0.5,
                           high=S2 + (S2 - S1)*0.5,
                           size=[N1,2])

    # Sampler #2: Dirichlet
    t2 = np.random.uniform(low=T0 - 0.5*(T - T0) , high=T,
                           size=[N2,1])
    s2 = np.random.uniform(low=S1 , high=S2  ,
                           size=[N2,2])
    s2[:,1]=0 #y=0
    
    # Sampler #3: Neumann
    t3 = np.random.uniform(low=T0 - 0.5*(T - T0) , high=T,
                           size=[N3,1])
    s3 = np.random.uniform(low=S1 - (S2 - S1)*0.5 , high=S2 + (S2 - S1)*0.5 ,
                           size=[N3,2])
    s3[:100,0]=0  #x=0
    s3[100:200,0]= 1 #x=1
    s3[200:,1]= 1 #y=1
    
    
    # Sampler #4: initial/terminal condition
    t4 = 0 * np.ones((N4,1)) #Initial condition
    s4 = np.random.uniform(low=S1, high=S2,
                           size=[N4,2])
    
    t1=torch.tensor(t1, dtype=torch.float32, requires_grad=True).cuda()
    s1=torch.tensor(s1, dtype=torch.float32, requires_grad=True).cuda()
    t2=torch.tensor(t2, dtype=torch.float32, requires_grad=True).cuda()
    s2=torch.tensor(s2, dtype=torch.float32, requires_grad=True).cuda()
    t3=torch.tensor(t3, dtype=torch.float32, requires_grad=True).cuda()
    s3=torch.tensor(s3, dtype=torch.float32, requires_grad=True).cuda()
    t4=torch.tensor(t4, dtype=torch.float32, requires_grad=True).cuda()
    s4=torch.tensor(s4, dtype=torch.float32, requires_grad=True).cuda()
    
    return (t1, s1, t2, s2, t3, s3, t4, s4)

In [6]:
def Loss(model, t1, x1, t2, x2, t3, x3, t4, x4):
    # Loss term #1: PDE
    U = model(t1, x1)
    U_t = torch.autograd.grad(U.sum(), t1, create_graph=True, retain_graph=True)[0]
    dU = torch.autograd.grad(U.sum(), x1, create_graph=True, retain_graph=True)[0]
    U_xx = torch.autograd.grad(dU[:,0].sum(), x1, create_graph = True, retain_graph=True)[0][:,0]
    U_yy = torch.autograd.grad(dU[:,1].sum(), x1, create_graph = True, retain_graph=True)[0][:,1]

    f = a*U_t + b*(U_xx + U_yy) + c*(U-Ta) + d*(U**4-Ta**4)
    L1 = torch.mean(torch.pow(f,2))

    # Loss term #2: Dirichlet boundary condition
    L2 = torch.mean(torch.pow(model(t2,x2)[:,0]-Tb,2))
   
    # Loss term #3: initial/terminal condition
    Un = model(t3,x3)
    dUn =  torch.autograd.grad(Un.sum(), x3, create_graph=True, retain_graph=True)[0]
    #GOT TO MAKE A VAR FOR NUMBER OF POINT PER SIDE TO HAVE A CORRECT IMPLEMENTATION !!!!!!
    dUn_normal = torch.cat((dUn[:200,0], dUn[200:,1]))
    L3 = torch.mean(torch.pow(dUn_normal,2))
    #L3 = torch.tensor(0.0)

    
    L4 = torch.mean(torch.pow((model(x4,t4)[:,0]-Ta),2))
    

    
    return L1, L2, L3, L4


Model init and training

In [7]:
model = DGM_net(3,1,8,200)
model.cuda()
opt = torch.optim.Adam(model.parameters(),lr = 0.005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(opt,gamma=0.99)

# Training parameters
steps_per_sample = 20
sampling_stages = 5000

In [8]:
def train_model(model, optimizer, scheduler, num_epochs=100):
    since = time.time()
    model.train()
  # Set model to training mode
    
    for epoch in range(num_epochs):
        t1, x1, t2, x2, t3, x3, t4, x4 = sampler(NS_1, NS_2, NS_3, NS_4) #
        scheduler.step()

        for _ in range(steps_per_sample) :

            # zero the parameter gradients
            optimizer.zero_grad()
        
            # forward
            L1, L2, L3, L4 = Loss(model, t1, x1, t2, x2, t3, x3, t4, x4)
    
            loss = L1 + L2 + L4  
            # backward + optimize
            loss.backward()
            optimizer.step()
            
        epoch +=1 
        if epoch % (num_epochs//num_epochs) == 0: print(f'epoch {epoch}, loss {loss.data}, L1 : {L1.data}, L2 : {L2.data}, L3 : {L3.data}, L4 : {L4.data}')
    time_elapsed = time.time() - since
    print(f"Training finished in {time_elapsed:.2f} for {num_epochs}.")
    print(f"The final loss value is {loss.data}")

In [9]:
train_model(model, opt, scheduler, sampling_stages)

epoch 1, loss 1988192.875, L1 : 933170.75, L2 : 989962.875, L3 : 0.06737013161182404, L4 : 65059.30859375
epoch 2, loss 2045878.75, L1 : 968860.125, L2 : 991026.375, L3 : 0.02315964549779892, L4 : 85992.2578125
epoch 3, loss 2034429.875, L1 : 955527.5625, L2 : 991211.0, L3 : 0.002334861783310771, L4 : 87691.359375
epoch 4, loss 2258178.5, L1 : 1175156.875, L2 : 992487.0, L3 : 0.0019701167475432158, L4 : 90534.5


KeyboardInterrupt: 

In [42]:
t1, x1, t2, x2, t3, x3, t4, x4 = sampler(NS_1, NS_2, NS_3, NS_4) 


In [43]:
x2

tensor([[0.6263, 1.0000],
        [0.1139, 1.0000],
        [0.7631, 1.0000],
        [0.6531, 1.0000],
        [0.2642, 1.0000],
        [0.0770, 1.0000],
        [0.4420, 1.0000],
        [0.6800, 1.0000],
        [0.8761, 1.0000],
        [0.7321, 1.0000],
        [0.9458, 1.0000],
        [0.1331, 1.0000],
        [0.0976, 1.0000],
        [0.6534, 1.0000],
        [0.1483, 1.0000],
        [0.7799, 1.0000],
        [0.5299, 1.0000],
        [0.6721, 1.0000],
        [0.4372, 1.0000],
        [0.0093, 1.0000],
        [0.8439, 1.0000],
        [0.0209, 1.0000],
        [0.0079, 1.0000],
        [0.1766, 1.0000],
        [0.8470, 1.0000],
        [0.2923, 1.0000],
        [0.2866, 1.0000],
        [0.4132, 1.0000],
        [0.6335, 1.0000],
        [0.4994, 1.0000],
        [0.1232, 1.0000],
        [0.4166, 1.0000],
        [0.8696, 1.0000],
        [0.9125, 1.0000],
        [0.3690, 1.0000],
        [0.7375, 1.0000],
        [0.6659, 1.0000],
        [0.1369, 1.0000],
        [0.0

In [ ]:
for param in model.parameters():
    print(param.grad.data.sum())

In [ ]:
for param in model.parameters():
  print(param.data)

In [ ]:
for param in model.parameters():
  print(param.data)

In [ ]:
xplot.reshape(-1,1)